In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# CV = 0: Best IoU: 0.83004
# CV = 2: Best IoU: 0.8243
# CV = 3: Best IoU: 0.821
# CV = 4: Best IoU: 0.8388

In [3]:
import gc
import pandas as pd
import numpy as np
import torch
import datetime
from tqdm import tqdm_notebook
from torch import nn
from torchvision import transforms
from cnn_finetune import make_model
from sklearn.model_selection import StratifiedKFold

import model4 as M
import unet_parts
from dataset import SegmentationDataset, SegmentationInferenceDataset
from data import *
from util import *
from loss import FocalLoss, dice_loss
from lovasz_loss import lovasz_hinge_flat, lovasz_hinge

In [4]:
torch.cuda.is_available()
device = torch.device('cuda')

In [5]:
# Settings
# ========

# Target k in KFold
CV = 0

# Skip pretraining
skip_pretraining = True

In [6]:
train_df, val_df = get_dfs_fold(k=CV)

train_dataset = SegmentationDataset(train_df, size=(128, 128),
                                    use_depth_channels=True,
                                    with_aux_label=False,
                                    as_aux_label='coverage_class',
                                    use_augmentation=True,
                                    mean_sub=False)
val_dataset = SegmentationInferenceDataset(val_df, input_size=(128, 128),
                                           use_depth_channels=True, with_aux_label=False, with_gt=True,
                                           mean_sub=False)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=2)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=8, shuffle=False)
print('Loaded dataset and created loader')

Use augmentations
Loaded dataset and created loader


# Training

In [7]:
gc.collect()
torch.cuda.empty_cache()

model = M.UNetRes34BilinearHcSCSEv5(n_classes=1)

if skip_pretraining:
    pretrained_model = model_dir / 'unet_res34_bilinear_hcscse_v5_kfold_cv{}_phase2_dict.model'.format(CV)
    W = torch.load(str(pretrained_model))
    model.load_state_dict(W)
    print('loaded: {}'.format(pretrained_model))
    
model = model.to(device)

optimizer = torch.optim.SGD(model.parameters(), lr=0.005, momentum=0.9)
#optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

Loaded pretrained resnet weights
loaded: D:\Users\ns\git_repos\kaggle-tgs-salt\models\unet_res34_bilinear_hcscse_v5_kfold_cv0_phase2_dict.model


In [8]:
def criterion_phase1(logit, target_pixel):
    segmentation_loss = F.binary_cross_entropy_with_logits(logit.view(-1), target_pixel.view(-1), size_average=True)
    #segmentation_dice_loss = dice_loss(logit, target_pixel)
    
    return segmentation_loss

In [9]:
def train_phase1(model, n_epoch, train_iter, val_iter):
    best_iou = 0.0
    n_stay = 0
    early_stopping_limit = 100
    
    for epoch in range(n_epoch):
        model.train()
        
        gc.collect()
        torch.cuda.empty_cache()
        
        total_loss = 0
        total_size = 0
        
        for batch_idx, (data, target) in enumerate(train_iter):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            
            # Forward
            logit = model(data)
            loss = criterion_phase1(logit, target)
            
            total_loss += loss.item()
            total_size += data.size(0)
            
            # Backward
            loss.backward()
            optimizer.step()
            
            if batch_idx % 50 == 0:
                now = datetime.datetime.now()
                print('[{}] Train Epoch: {} [{}/{} ({:.0f}%)]\tAverage loss: {:.6f}'.format(
                    now,
                    epoch, batch_idx * len(data), len(train_iter.dataset),
                    100. * batch_idx / len(train_iter), total_loss / total_size))
                
        gc.collect()
        torch.cuda.empty_cache()
        
        with torch.no_grad():
            iou = evaluate(model, val_iter, device=device, use_sigmoid=True, threshold=0.5)
        print('[{}] Train Epoch: {}\tIoU: {:.6f}'.format(now, epoch, iou))
        
        if best_iou < iou:
            best_iou = iou
            save_model(model, f'unet_res34_bilinear_hcscse_v5_kfold_cv{CV}_phase1')
            print('Saved model at {} (IoU: {})'.format(epoch, iou))
            n_stay = 0
        else:
            n_stay += 1
        
        if n_stay >= early_stopping_limit:
            print('Early stopping at {} (Best IoU: {})'.format(epoch, best_iou))
            break
            
    return model

In [10]:
if not skip_pretraining:
    model = train_phase1(model, 50, train_loader, val_loader)

D:\Users\ns\Anaconda3\envs\chainer\lib\site-packages\torch\nn\functional.py:1890: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")
D:\Users\ns\Anaconda3\envs\chainer\lib\site-packages\torch\nn\functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='elementwise_mean' instead.
  warnings.warn(warning.format(ret))


[2018-10-20 04:42:01.506574] Train Epoch: 0 [0/3198 (0%)]	Average loss: 0.042890
[2018-10-20 04:42:26.673299] Train Epoch: 0 [800/3198 (25%)]	Average loss: 0.030348
[2018-10-20 04:42:51.917267] Train Epoch: 0 [1600/3198 (50%)]	Average loss: 0.026296
[2018-10-20 04:43:17.217495] Train Epoch: 0 [2400/3198 (75%)]	Average loss: 0.024636
[2018-10-20 04:43:17.217495] Train Epoch: 0	IoU: 0.600748
Saved model at 0 (IoU: 0.6007481296758105)
[2018-10-20 04:44:01.790903] Train Epoch: 1 [0/3198 (0%)]	Average loss: 0.011150
[2018-10-20 04:44:24.488541] Train Epoch: 1 [800/3198 (25%)]	Average loss: 0.018027
[2018-10-20 04:44:47.191454] Train Epoch: 1 [1600/3198 (50%)]	Average loss: 0.017507
[2018-10-20 04:45:09.919698] Train Epoch: 1 [2400/3198 (75%)]	Average loss: 0.017149
[2018-10-20 04:45:09.919698] Train Epoch: 1	IoU: 0.632668
Saved model at 1 (IoU: 0.6326683291770573)
[2018-10-20 04:45:43.970328] Train Epoch: 2 [0/3198 (0%)]	Average loss: 0.012759
[2018-10-20 04:46:06.675619] Train Epoch: 2 [80

[2018-10-20 05:15:29.340939] Train Epoch: 19	IoU: 0.760973
[2018-10-20 05:16:02.452547] Train Epoch: 20 [0/3198 (0%)]	Average loss: 0.012190
[2018-10-20 05:16:24.923724] Train Epoch: 20 [800/3198 (25%)]	Average loss: 0.009313
[2018-10-20 05:16:47.405835] Train Epoch: 20 [1600/3198 (50%)]	Average loss: 0.008226
[2018-10-20 05:17:09.900300] Train Epoch: 20 [2400/3198 (75%)]	Average loss: 0.008693
[2018-10-20 05:17:09.900300] Train Epoch: 20	IoU: 0.750000
[2018-10-20 05:17:43.077762] Train Epoch: 21 [0/3198 (0%)]	Average loss: 0.013295
[2018-10-20 05:18:05.557627] Train Epoch: 21 [800/3198 (25%)]	Average loss: 0.008143
[2018-10-20 05:18:28.058656] Train Epoch: 21 [1600/3198 (50%)]	Average loss: 0.008179
[2018-10-20 05:18:50.556129] Train Epoch: 21 [2400/3198 (75%)]	Average loss: 0.007981
[2018-10-20 05:18:50.556129] Train Epoch: 21	IoU: 0.747880
[2018-10-20 05:19:23.651324] Train Epoch: 22 [0/3198 (0%)]	Average loss: 0.020440
[2018-10-20 05:19:46.121368] Train Epoch: 22 [800/3198 (25%)]	A

[2018-10-20 05:49:02.106537] Train Epoch: 39	IoU: 0.788279
[2018-10-20 05:49:35.257394] Train Epoch: 40 [0/3198 (0%)]	Average loss: 0.003264
[2018-10-20 05:49:57.992705] Train Epoch: 40 [800/3198 (25%)]	Average loss: 0.005381
[2018-10-20 05:50:20.699446] Train Epoch: 40 [1600/3198 (50%)]	Average loss: 0.005336
[2018-10-20 05:50:43.451206] Train Epoch: 40 [2400/3198 (75%)]	Average loss: 0.005277
[2018-10-20 05:50:43.451206] Train Epoch: 40	IoU: 0.759975
[2018-10-20 05:51:16.814335] Train Epoch: 41 [0/3198 (0%)]	Average loss: 0.002523
[2018-10-20 05:51:39.281357] Train Epoch: 41 [800/3198 (25%)]	Average loss: 0.004984
[2018-10-20 05:52:01.798846] Train Epoch: 41 [1600/3198 (50%)]	Average loss: 0.004999
[2018-10-20 05:52:24.275493] Train Epoch: 41 [2400/3198 (75%)]	Average loss: 0.005230
[2018-10-20 05:52:24.275493] Train Epoch: 41	IoU: 0.778554
[2018-10-20 05:52:57.424439] Train Epoch: 42 [0/3198 (0%)]	Average loss: 0.003957
[2018-10-20 05:53:19.926602] Train Epoch: 42 [800/3198 (25%)]	A

In [8]:
def criterion_phase2(logit, target_pixel):
    segmentation_loss = lovasz_hinge(logit.squeeze(), target_pixel.squeeze())
    
    return segmentation_loss

In [12]:
def train_phase2(model, n_epoch, train_iter, val_iter):
    best_iou = 0.0
    n_stay = 0
    early_stopping_limit = 100
    reduce_limit = 10
    min_lr = 0.000001
    base_lr = 0.005
    current_lr = base_lr
    
    for epoch in range(n_epoch):
        model.train()
        
        gc.collect()
        torch.cuda.empty_cache()
        
        total_loss = 0
        total_size = 0
        
        for batch_idx, (data, target) in enumerate(train_iter):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            
            # Forward
            logit = model(data)
            loss = criterion_phase2(logit, target)
            
            total_loss += loss.item()
            total_size += data.size(0)
            
            # Backward
            loss.backward()
            optimizer.step()
            
            if batch_idx % 50 == 0:
                now = datetime.datetime.now()
                print('[{}] Train Epoch: {} [{}/{} ({:.0f}%)]\tAverage loss: {:.6f}'.format(
                    now,
                    epoch, batch_idx * len(data), len(train_iter.dataset),
                    100. * batch_idx / len(train_iter), total_loss / total_size))
                
        gc.collect()
        torch.cuda.empty_cache()
        
        with torch.no_grad():
            iou = evaluate(model, val_iter, device=device, use_sigmoid=False, threshold=0.0)
        print('[{}] Train Epoch: {}\tIoU: {:.6f}'.format(now, epoch, iou))
        
        if best_iou < iou:
            best_iou = iou
            save_model(model, f'unet_res34_bilinear_hcscse_v5_kfold_cv{CV}_phase2')
            print('Saved model at {} (IoU: {})'.format(epoch, iou))
            n_stay = 0
        else:
            n_stay += 1
            
        if n_stay >= early_stopping_limit:
            print('Early stopping at {} (Best IoU: {})'.format(epoch, best_iou))
            break
            
        if n_stay >= reduce_limit:
            current_lr = 0.5 * current_lr
            for g in optimizer.param_groups:
                g['lr'] = current_lr
            n_stay = 0
            print('Reduce lr at {} (to: {})'.format(epoch, current_lr))
        
    return model

In [13]:
model = train_phase2(model, 80, train_loader, val_loader)

D:\Users\ns\Anaconda3\envs\chainer\lib\site-packages\torch\nn\functional.py:1890: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


[2018-10-20 06:06:19.926319] Train Epoch: 0 [0/3198 (0%)]	Average loss: 0.069029
[2018-10-20 06:06:44.424870] Train Epoch: 0 [800/3198 (25%)]	Average loss: 0.077270
[2018-10-20 06:07:08.941718] Train Epoch: 0 [1600/3198 (50%)]	Average loss: 0.079190
[2018-10-20 06:07:33.407687] Train Epoch: 0 [2400/3198 (75%)]	Average loss: 0.076789
[2018-10-20 06:07:33.407687] Train Epoch: 0	IoU: 0.732918
Saved model at 0 (IoU: 0.732917705735661)
[2018-10-20 06:08:09.453555] Train Epoch: 1 [0/3198 (0%)]	Average loss: 0.035444
[2018-10-20 06:08:33.925504] Train Epoch: 1 [800/3198 (25%)]	Average loss: 0.072170
[2018-10-20 06:08:58.438364] Train Epoch: 1 [1600/3198 (50%)]	Average loss: 0.072789
[2018-10-20 06:09:22.918983] Train Epoch: 1 [2400/3198 (75%)]	Average loss: 0.073307
[2018-10-20 06:09:22.918983] Train Epoch: 1	IoU: 0.736409
Saved model at 1 (IoU: 0.7364089775561098)
[2018-10-20 06:09:58.540353] Train Epoch: 2 [0/3198 (0%)]	Average loss: 0.062121
[2018-10-20 06:10:23.036499] Train Epoch: 2 [800

[2018-10-20 06:41:56.512987] Train Epoch: 19 [2400/3198 (75%)]	Average loss: 0.054328
[2018-10-20 06:41:56.512987] Train Epoch: 19	IoU: 0.803990
Saved model at 19 (IoU: 0.8039900249376559)
[2018-10-20 06:42:31.809889] Train Epoch: 20 [0/3198 (0%)]	Average loss: 0.063639
[2018-10-20 06:42:56.238162] Train Epoch: 20 [800/3198 (25%)]	Average loss: 0.047478
[2018-10-20 06:43:20.660437] Train Epoch: 20 [1600/3198 (50%)]	Average loss: 0.049668
[2018-10-20 06:43:45.108958] Train Epoch: 20 [2400/3198 (75%)]	Average loss: 0.048727
[2018-10-20 06:43:45.108958] Train Epoch: 20	IoU: 0.798005
[2018-10-20 06:44:20.036359] Train Epoch: 21 [0/3198 (0%)]	Average loss: 0.041382
[2018-10-20 06:44:44.522651] Train Epoch: 21 [800/3198 (25%)]	Average loss: 0.049411
[2018-10-20 06:45:08.982472] Train Epoch: 21 [1600/3198 (50%)]	Average loss: 0.050050
[2018-10-20 06:45:33.448689] Train Epoch: 21 [2400/3198 (75%)]	Average loss: 0.049379
[2018-10-20 06:45:33.448689] Train Epoch: 21	IoU: 0.792145
[2018-10-20 06:

[2018-10-20 07:18:04.942216] Train Epoch: 39 [2400/3198 (75%)]	Average loss: 0.040706
[2018-10-20 07:18:04.942216] Train Epoch: 39	IoU: 0.810224
[2018-10-20 07:18:39.661856] Train Epoch: 40 [0/3198 (0%)]	Average loss: 0.028177
[2018-10-20 07:19:04.160488] Train Epoch: 40 [800/3198 (25%)]	Average loss: 0.039861
[2018-10-20 07:19:28.622219] Train Epoch: 40 [1600/3198 (50%)]	Average loss: 0.037831
[2018-10-20 07:19:53.099757] Train Epoch: 40 [2400/3198 (75%)]	Average loss: 0.039786
[2018-10-20 07:19:53.099757] Train Epoch: 40	IoU: 0.813716
[2018-10-20 07:20:27.816855] Train Epoch: 41 [0/3198 (0%)]	Average loss: 0.045928
[2018-10-20 07:20:52.364209] Train Epoch: 41 [800/3198 (25%)]	Average loss: 0.037221
[2018-10-20 07:21:16.819867] Train Epoch: 41 [1600/3198 (50%)]	Average loss: 0.038629
[2018-10-20 07:21:41.347350] Train Epoch: 41 [2400/3198 (75%)]	Average loss: 0.040961
[2018-10-20 07:21:41.347350] Train Epoch: 41	IoU: 0.817332
Saved model at 41 (IoU: 0.8173316708229427)
[2018-10-20 07:

KeyboardInterrupt: 

In [9]:
def train_phase3(model, n_epoch, train_iter, val_iter):
    best_iou = 0.818703
    n_stay = 0
    early_stopping_limit = 100
    reduce_limit = 10
    min_lr = 0.000001
    base_lr = 0.005
    current_lr = base_lr
    
    for g in optimizer.param_groups:
        g['lr'] = current_lr
    n_stay = 0
    print('Reduce lr at 0 (to: {})'.format(current_lr))
    
    for epoch in range(n_epoch):
        model.train()
        
        gc.collect()
        torch.cuda.empty_cache()
        
        total_loss = 0
        total_size = 0
        
        for batch_idx, (data, target) in enumerate(train_iter):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            
            # Forward
            logit = model(data)
            loss = criterion_phase2(logit, target)
            
            total_loss += loss.item()
            total_size += data.size(0)
            
            # Backward
            loss.backward()
            optimizer.step()
            
            if batch_idx % 50 == 0:
                now = datetime.datetime.now()
                print('[{}] Train Epoch: {} [{}/{} ({:.0f}%)]\tAverage loss: {:.6f}'.format(
                    now,
                    epoch, batch_idx * len(data), len(train_iter.dataset),
                    100. * batch_idx / len(train_iter), total_loss / total_size))
                
        gc.collect()
        torch.cuda.empty_cache()
        
        with torch.no_grad():
            iou = evaluate(model, val_iter, device=device, use_sigmoid=False, threshold=0.0)
        print('[{}] Train Epoch: {}\tIoU: {:.6f}'.format(now, epoch, iou))
        
        if best_iou < iou:
            best_iou = iou
            save_model(model, f'unet_res34_bilinear_hcscse_v5_kfold_cv{CV}_phase2')
            print('Saved model at {} (IoU: {})'.format(epoch, iou))
            n_stay = 0
        else:
            n_stay += 1
            
        if n_stay >= early_stopping_limit:
            print('Early stopping at {} (Best IoU: {})'.format(epoch, best_iou))
            break
            
        if n_stay >= reduce_limit:
            current_lr = 0.5 * current_lr
            for g in optimizer.param_groups:
                g['lr'] = current_lr
            n_stay = 0
            print('Reduce lr at {} (to: {})'.format(epoch, current_lr))
        
    return model

In [10]:
model = train_phase3(model, 80, train_loader, val_loader)

Reduce lr at 0 (to: 0.005)


D:\Users\ns\Anaconda3\envs\chainer\lib\site-packages\torch\nn\functional.py:1890: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


[2018-10-20 07:39:51.288665] Train Epoch: 0 [0/3198 (0%)]	Average loss: 0.040267
[2018-10-20 07:40:21.852726] Train Epoch: 0 [800/3198 (25%)]	Average loss: 0.039165
[2018-10-20 07:40:52.383510] Train Epoch: 0 [1600/3198 (50%)]	Average loss: 0.036547
[2018-10-20 07:41:22.862354] Train Epoch: 0 [2400/3198 (75%)]	Average loss: 0.035856
[2018-10-20 07:41:22.862354] Train Epoch: 0	IoU: 0.823192
Saved model at 0 (IoU: 0.8231920199501247)
[2018-10-20 07:42:06.058323] Train Epoch: 1 [0/3198 (0%)]	Average loss: 0.069818
[2018-10-20 07:42:30.837094] Train Epoch: 1 [800/3198 (25%)]	Average loss: 0.037938
[2018-10-20 07:42:55.571951] Train Epoch: 1 [1600/3198 (50%)]	Average loss: 0.037050
[2018-10-20 07:43:20.339146] Train Epoch: 1 [2400/3198 (75%)]	Average loss: 0.036087
[2018-10-20 07:43:20.339146] Train Epoch: 1	IoU: 0.830050
Saved model at 1 (IoU: 0.8300498753117208)
[2018-10-20 07:43:56.482702] Train Epoch: 2 [0/3198 (0%)]	Average loss: 0.017208
[2018-10-20 07:44:21.289407] Train Epoch: 2 [80

KeyboardInterrupt: 

In [ ]:
val_check_dataset = SegmentationInferenceDataset(val_df, input_size=(128, 128),
                                                 with_gt=True, with_raw_input=True, use_depth_channels=True,
                                                 mean_sub=True)
val_check_loader = torch.utils.data.DataLoader(val_check_dataset, batch_size=8, shuffle=False)
val_check_iter = iter(val_check_loader)

In [ ]:
model.eval()
im, x, t = next(val_check_iter)
pred = predict(model, x, device, with_tta=True, use_sigmoid=True, threshold=0.5)
show_prediction(im, pred, t)

In [ ]:
model.eval()
im, x, t = next(val_check_iter)
pred = predict(model, x, device, with_tta=True, use_sigmoid=False, threshold=0.0)
show_prediction(im, pred, t)

In [ ]:
model.eval()
im, x, t = next(val_check_iter)
pred = predict(model, x, device, with_tta=True, use_sigmoid=False, threshold=0.0)
show_prediction(im, pred, t)

In [ ]:
model.eval()
im, x, t = next(val_check_iter)
pred = predict(model, x, device, with_tta=True, use_sigmoid=False, threshold=0.0)
show_prediction(im, pred, t)